In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm.autonotebook import tqdm

In [ ]:
df_train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
df_train.head()

In [ ]:
df_test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
df_test

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def text_cleaning(text):
    text = ''.join([k for k in text if k not in string.punctuation])
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    text = re.sub(' +', ' ', text)
    return text

In [ ]:
train_sentences = df_train["excerpt"].tolist()
train_sentences[1]

In [ ]:
test_sentences = df_test["excerpt"].tolist()
test_sentences[1]

In [ ]:
train_labels = df_train["target"].tolist()

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences) #sentences is list
word_index = tokenizer.word_index
print(len(word_index))

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
print(reverse_word_index[9])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
print(decode_sentence(train_padded[0]))
print(train_sentences[0])
print(train_labels[0])

In [ ]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
train_padded = np.asarray(train_padded)
train_labels = np.asarray(train_labels)
test_padded = np.asarray(test_padded)
#test_labels = np.array(test_labels)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_padded,train_labels,test_size=0.2, random_state=0)

In [ ]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from sklearn.svm import SVR
clf = SVR(kernel="linear")
clf.fit(X_train, y_train)

print(clf.predict(X_test))
predict = clf.predict(test_padded)
predict

In [ ]:
#predict = model.predict(test_padded)
#predict

In [ ]:
sub_sample = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")
sub_sample['target'] = predict
sub_sample.to_csv('submission.csv', index=False)
pd.set_option("max_colwidth",400)
sub_sample